#### Title: dfg2_wweia_diet_recalls
#### Purpose: link DFG2 data with WWEIA ingredients; yeilding intakes for all glycan features across WWEIA 01-18
#### Date: February 14, 2024
#### Author: Jules Larke

In [1]:
# Import modules
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

In [2]:
# Load data
dfg2_data = pd.read_csv('../data/wweia/dfg2_wweia_data.csv')
recalls = pd.read_csv('/Users/jules.larke/work/github/wweia_ingredients/data/04/wweia_all_recalls.txt', sep='\t')

In [3]:
# Select variables of interest for DFG2
recalls_select = recalls[['SEQN', 'RIAGENDR', 'RIDAGEYR', 'RIDRETH1', 'DMDEDUC2', 'DMDEDUC3', 'ingred_code', 'ingred_desc', 'Carbohydrate', 'Fiber, total dietary', 'Sugars, total', 'Energy', 'Alcohol', 'Ingred_consumed_g', 'diet_wts']]

In [4]:
# subset to include 18+ y/o
adults = recalls_select[recalls_select['RIDAGEYR'] > 17]

In [5]:
adults.rename(columns={'RIDAGEYR': 'Age'},inplace=True)
adults.rename(columns={'RIAGENDR': 'Sex'},inplace=True)
adults.rename(columns={'RIDRETH1': 'Ethnicity'},inplace=True)

/var/folders/28/kcjp2s310sx44jpn8yqs8y9c929k6r/T/ipykernel_96522/3960642062.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adults.rename(columns={'RIDAGEYR': 'Age'},inplace=True)
/var/folders/28/kcjp2s310sx44jpn8yqs8y9c929k6r/T/ipykernel_96522/3960642062.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adults.rename(columns={'RIAGENDR': 'Sex'},inplace=True)
/var/folders/28/kcjp2s310sx44jpn8yqs8y9c929k6r/T/ipykernel_96522/3960642062.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/u

### Education

In [6]:
# value is NA, replace with 'unknown'
adults.loc[adults.SEQN==16247, 'DMDEDUC2'] = 9

In [7]:
# recode edu levels for age 19 and under
adults['DMDEDUC3'] = adults['DMDEDUC3'].replace([13, 14, 15], 'high school graduate or equivalent')
adults['DMDEDUC3'] = adults['DMDEDUC3'].replace([9, 10, 11, 12, 66, 99], 'less than high school graduate')

/var/folders/28/kcjp2s310sx44jpn8yqs8y9c929k6r/T/ipykernel_96522/2139188517.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adults['DMDEDUC3'] = adults['DMDEDUC3'].replace([13, 14, 15], 'high school graduate or equivalent')
/var/folders/28/kcjp2s310sx44jpn8yqs8y9c929k6r/T/ipykernel_96522/2139188517.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adults['DMDEDUC3'] = adults['DMDEDUC3'].replace([9, 10, 11, 12, 66, 99], 'less than high school graduate')


In [8]:
# recode edu levels for age 20 and older
adults['DMDEDUC2'] = adults['DMDEDUC2'].replace([1, 2], 'less than high school graduate')
adults['DMDEDUC2'] = adults['DMDEDUC2'].replace(3, 'high school graduate or equivalent')
adults['DMDEDUC2'] = adults['DMDEDUC2'].replace(4, 'some college')
adults['DMDEDUC2'] = adults['DMDEDUC2'].replace(5, 'college graduate')
adults['DMDEDUC2'] = adults['DMDEDUC2'].replace([7, 9], 'unknown')

/var/folders/28/kcjp2s310sx44jpn8yqs8y9c929k6r/T/ipykernel_96522/1115660030.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adults['DMDEDUC2'] = adults['DMDEDUC2'].replace([1, 2], 'less than high school graduate')
/var/folders/28/kcjp2s310sx44jpn8yqs8y9c929k6r/T/ipykernel_96522/1115660030.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adults['DMDEDUC2'] = adults['DMDEDUC2'].replace(3, 'high school graduate or equivalent')
/var/folders/28/kcjp2s310sx44jpn8yqs8y9c929k6r/T/ipykernel_96522/1115660030.p

In [9]:
# create single feature for education including all participants
adults['education'] = adults.filter(like='DMDEDUC').ffill(axis=1).iloc[:,-1].copy()

/var/folders/28/kcjp2s310sx44jpn8yqs8y9c929k6r/T/ipykernel_96522/2443542517.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adults['education'] = adults.filter(like='DMDEDUC').ffill(axis=1).iloc[:,-1].copy()


In [10]:
adults = adults.drop(columns=['DMDEDUC2', 'DMDEDUC3'])

### BMI

In [11]:
# Exam data - Body Measures (BMI: BMXBMI)
bmx_B = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2001-2002/BMX_B.XPT', format='xport', encoding='utf-8')
bmx_C = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2003-2004/BMX_C.XPT', format='xport', encoding='utf-8')
bmx_D = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2005-2006/BMX_D.XPT', format='xport', encoding='utf-8')
bmx_E = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2007-2008/BMX_E.XPT', format='xport', encoding='utf-8')
bmx_F = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2009-2010/BMX_F.XPT', format='xport', encoding='utf-8')
bmx_G = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2011-2012/BMX_G.XPT', format='xport', encoding='utf-8')
bmx_H = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2013-2014/BMX_H.XPT', format='xport', encoding='utf-8')
bmx_I = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2015-2016/BMX_I.XPT', format='xport', encoding='utf-8')
bmx_J = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/BMX_J.XPT', format='xport', encoding='utf-8')

In [12]:
bmx_B = bmx_B[['SEQN', 'BMXBMI', 'BMXWAIST']]
bmx_C = bmx_C[['SEQN', 'BMXBMI', 'BMXWAIST']]
bmx_D = bmx_D[['SEQN', 'BMXBMI', 'BMXWAIST']]
bmx_E = bmx_E[['SEQN', 'BMXBMI', 'BMXWAIST']]
bmx_F = bmx_F[['SEQN', 'BMXBMI', 'BMXWAIST']]
bmx_G = bmx_G[['SEQN', 'BMXBMI', 'BMXWAIST']]
bmx_H = bmx_H[['SEQN', 'BMXBMI', 'BMXWAIST']]
bmx_I = bmx_I[['SEQN', 'BMXBMI', 'BMXWAIST']]
bmx_J = bmx_J[['SEQN', 'BMXBMI', 'BMXWAIST']]

bmi = pd.concat([bmx_B, bmx_C, bmx_D, bmx_E, bmx_F, bmx_G, bmx_H, bmx_I, bmx_J])
bmi.rename(columns={'BMXBMI': 'BMI', 'BMXWAIST': 'WC'}, inplace=True)

In [13]:
adults = adults.merge(bmi, on='SEQN', how='left')

### Impute BMI from WC for those who have WC measured (197)

In [14]:
no_bmi = adults[adults['BMI'].isnull()]

In [15]:
no_bmi = no_bmi[['SEQN', 'Sex', 'BMI', 'WC']]

In [16]:
no_bmi = no_bmi.drop_duplicates(subset=['SEQN'])

In [17]:
no_bmi = no_bmi[~no_bmi['WC'].isnull()]

In [18]:
no_bmi_m = no_bmi[no_bmi['Sex']=='Male']
no_bmi_f = no_bmi[no_bmi['Sex']=='Female']

In [19]:
bmi_wc = adults[['SEQN', 'Sex', 'BMI', 'WC']]

In [20]:
bmi_wc = bmi_wc.drop_duplicates(subset=['SEQN'])

In [21]:
bmi_wc = bmi_wc.dropna(subset=['BMI', 'WC'])

In [22]:
bmi_wc_m = bmi_wc[bmi_wc['Sex']=='Male']

In [23]:
bmi_wc_f = bmi_wc[bmi_wc['Sex']=='Female']

In [24]:
x_m = np.array(bmi_wc_m['WC']).reshape((-1, 1))
y_m = np.array(bmi_wc_m['BMI']).reshape((-1, 1))
z_m = np.array(no_bmi_m['WC']).reshape((-1, 1))

x_f = np.array(bmi_wc_f['WC']).reshape((-1, 1))
y_f = np.array(bmi_wc_f['BMI']).reshape((-1, 1))
z_f = np.array(no_bmi_f['WC']).reshape((-1, 1))

In [25]:
model_m = LinearRegression().fit(x_m, y_m)

In [26]:
r_sq = model_m.score(x_m, y_m)
print(f"coefficient of determination: {r_sq}")
print(f"intercept: {model_m.intercept_}")
print(f"slope: {model_m.coef_}")

coefficient of determination: 0.8461023921101636
intercept: [-5.87308117]
slope: [[0.34191912]]


In [27]:
y_pred_m = model_m.predict(z_m)
print(f"predicted response:\n{y_pred_m}")

predicted response:
[[28.01110387]
 [26.81438694]
 [27.15630606]
 [24.11322587]
 [29.82327521]
 [35.6359003 ]
 [25.61767001]
 [27.73756857]
 [30.47292155]
 [31.80640612]
 [28.01110387]
 [29.54973992]
 [26.09635678]
 [18.09544931]
 [30.47292155]
 [24.04484205]
 [29.58393183]
 [22.19847878]
 [22.91650894]
 [24.7628722 ]
 [27.01953841]
 [24.86544794]
 [27.73756857]
 [21.00176186]
 [38.7131724 ]
 [32.72958775]
 [38.09771798]
 [25.92539722]
 [31.12256788]
 [21.00176186]
 [22.67716556]
 [36.93519296]
 [33.55019365]
 [30.71226493]
 [28.25044725]
 [26.60923546]
 [34.50756719]
 [26.81438694]
 [25.51509427]
 [24.4893369 ]
 [26.57504355]
 [32.69539584]
 [28.45559872]
 [31.66963848]
 [30.40453772]
 [22.67716556]
 [33.78953703]
 [28.59236637]
 [25.5834781 ]
 [32.0115576 ]
 [21.1385295 ]
 [30.3361539 ]
 [31.53287083]
 [28.69494211]
 [26.60923546]
 [31.7380223 ]
 [34.98625396]
 [30.81484067]
 [29.17362888]
 [33.61857747]
 [29.61812374]
 [23.6345391 ]
 [48.04756444]
 [27.36145753]
 [28.07948769]
 [32.

In [28]:
model_f = LinearRegression().fit(x_f, y_f)

In [29]:
r_sq = model_f.score(x_f, y_f)
print(f"coefficient of determination: {r_sq}")
print(f"intercept: {model_f.intercept_}")
print(f"slope: {model_f.coef_}")

coefficient of determination: 0.8384576719614756
intercept: [-9.98255724]
slope: [[0.40583216]]


In [30]:
y_pred_f = model_m.predict(z_f)
print(f"predicted response:\n{y_pred_f}")

predicted response:
[[21.99332731]
 [30.02842669]
 [21.61721628]
 [35.32817309]
 [30.71226493]
 [20.72822656]
 [22.36943835]
 [20.557267  ]
 [28.18206343]
 [43.53423202]
 [32.52443628]
 [21.00176186]
 [28.14787151]
 [28.79751785]
 [35.73847603]
 [20.86499421]
 [25.72024575]
 [41.38014155]
 [28.90009358]
 [31.05418405]
 [31.7380223 ]
 [27.29307371]
 [32.07994142]
 [21.48044863]
 [34.98625396]
 [41.58529303]
 [27.49822518]
 [35.22559735]
 [37.65322312]
 [33.9604966 ]
 [22.33524643]
 [30.19938625]
 [27.97691195]
 [26.30150825]
 [27.08792223]
 [31.08837597]
 [29.41297227]
 [18.91605521]
 [17.10388386]
 [41.03822243]
 [31.60125465]
 [36.62746575]
 [28.48979064]
 [29.515548  ]
 [29.58393183]
 [22.19847878]
 [30.13100242]
 [32.04574951]
 [22.43782217]
 [21.07014568]
 [27.53241709]
 [29.68650757]
 [28.79751785]
 [26.13054869]
 [29.54973992]
 [32.04574951]
 [28.96847741]
 [29.03686123]
 [25.0364075 ]
 [19.77085301]
 [32.31928481]
 [23.22423615]
 [31.05418405]
 [45.4147872 ]
 [35.0888297 ]
 [21.

In [31]:
no_bmi_m['BMI'] = y_pred_m.flatten()

/var/folders/28/kcjp2s310sx44jpn8yqs8y9c929k6r/T/ipykernel_96522/2323749325.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  no_bmi_m['BMI'] = y_pred_m.flatten()


In [32]:
no_bmi_f['BMI'] = y_pred_f.flatten()

/var/folders/28/kcjp2s310sx44jpn8yqs8y9c929k6r/T/ipykernel_96522/4169922082.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  no_bmi_f['BMI'] = y_pred_f.flatten()


In [33]:
no_bmi = pd.concat([no_bmi_m, no_bmi_f])

In [34]:
adults.set_index('SEQN', inplace=True)

In [35]:
no_bmi.set_index('SEQN', inplace=True)

In [36]:
adults['BMI'].update(no_bmi['BMI'])

In [37]:
adults.dropna(subset='BMI', inplace=True)

In [38]:
adults.reset_index(inplace=True)

### Diabetes

In [39]:
#Lab data - fasting plasma insulin

fgi_B = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2001-2002/L10AM_B.XPT', format='xport', encoding='utf-8')
fgi_C = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2003-2004/L10AM_C.XPT', format='xport', encoding='utf-8')
fgi_D = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2005-2006/GLU_D.XPT', format='xport', encoding='utf-8')
fgi_E = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2007-2008/GLU_E.XPT', format='xport', encoding='utf-8')
fgi_F = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2009-2010/GLU_F.XPT', format='xport', encoding='utf-8')
fgi_G = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2011-2012/GLU_G.XPT', format='xport', encoding='utf-8')
fg_H = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2013-2014/GLU_H.XPT', format='xport', encoding='utf-8')
fi_H = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2013-2014/INS_H.XPT', format='xport', encoding='utf-8')
fg_I = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2015-2016/GLU_I.XPT', format='xport', encoding='utf-8')
fi_I = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2015-2016/INS_I.XPT', format='xport', encoding='utf-8')
fg_J = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/GLU_J.XPT', format='xport', encoding='utf-8')
fi_J = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/INS_J.XPT', format='xport', encoding='utf-8')

In [40]:
fgi_B = fgi_B[['SEQN', 'LBXGLU', 'LBXIN']]
fgi_C = fgi_C[['SEQN', 'LBXGLU', 'LBXIN']]
fgi_D = fgi_D[['SEQN', 'LBXGLU', 'LBXIN']]
fgi_E = fgi_E[['SEQN', 'LBXGLU', 'LBXIN']]
fgi_F = fgi_F[['SEQN', 'LBXGLU', 'LBXIN']]
fgi_G = fgi_G[['SEQN', 'LBXGLU', 'LBXIN']]

fg_HIJ = pd.concat([fg_H, fg_I, fg_J])
fi_HIJ = pd.concat([fi_H, fi_I, fi_J])
fg_HIJ = fg_HIJ[['SEQN', 'LBXGLU']]
fi_HIJ = fi_HIJ[['SEQN', 'LBXIN']]
fgi_HIJ = fg_HIJ.merge(fi_HIJ, on='SEQN', how='left')

fgi = pd.concat([fgi_B, fgi_C, fgi_D, fgi_E, fgi_F, fgi_G, fgi_HIJ])
fgi.rename(columns={'LBXGLU':'fasting_glc_mg_dL', 'LBXIN':'fasting_ins_uU_mL'}, inplace=True)

In [41]:
adults = adults.merge(fgi, on='SEQN', how='left')

In [42]:
adults['diabetes_fasting_glc'] = np.where(adults['fasting_glc_mg_dL'] >= 126, 'yes', 'no')

In [43]:
#Lab data - glycohemoglobic (hba1c)

gh_B = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2001-2002/L10_B.XPT', format='xport', encoding='utf-8')
gh_C = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2003-2004/L10_C.XPT', format='xport', encoding='utf-8')
gh_D = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2005-2006/GHB_D.XPT', format='xport', encoding='utf-8')
gh_E = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2007-2008/GHB_E.XPT', format='xport', encoding='utf-8')
gh_F = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2009-2010/GHB_F.XPT', format='xport', encoding='utf-8')
gh_G = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2011-2012/GHB_G.XPT', format='xport', encoding='utf-8')
gh_H = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2013-2014/GHB_H.XPT', format='xport', encoding='utf-8')
gh_I = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2015-2016/GHB_I.XPT', format='xport', encoding='utf-8')
gh_J = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/GHB_J.XPT', format='xport', encoding='utf-8')

In [44]:
gh_B = gh_B[['SEQN', 'LBXGH']]
gh_C = gh_C[['SEQN', 'LBXGH']]
gh_D = gh_D[['SEQN', 'LBXGH']]
gh_E = gh_E[['SEQN', 'LBXGH']]
gh_F = gh_F[['SEQN', 'LBXGH']]
gh_G = gh_G[['SEQN', 'LBXGH']]
gh_H = gh_H[['SEQN', 'LBXGH']]
gh_I = gh_I[['SEQN', 'LBXGH']]
gh_J = gh_J[['SEQN', 'LBXGH']]

gh = pd.concat([gh_B, gh_C, gh_D, gh_E, gh_F, gh_G, gh_H, gh_I, gh_J])
gh.rename(columns={'LBXGH':'hba1c_percent'}, inplace=True)

In [45]:
adults = adults.merge(gh, on='SEQN', how='left')

In [46]:
adults['diabetes_hba1c'] = np.where(adults['hba1c_percent'] >= 6.5, 'yes', 'no')

In [47]:
#Questionnaire data - taking insulin or glucose lowering meds

diq_B = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2001-2002/DIQ_B.XPT', format='xport', encoding='utf-8')
diq_C = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2003-2004/DIQ_C.XPT', format='xport', encoding='utf-8')
diq_D = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2005-2006/DIQ_D.XPT', format='xport', encoding='utf-8')
diq_E = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2007-2008/DIQ_E.XPT', format='xport', encoding='utf-8')
diq_F = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2009-2010/DIQ_F.XPT', format='xport', encoding='utf-8')
diq_G = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2011-2012/DIQ_G.XPT', format='xport', encoding='utf-8')
diq_H = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2013-2014/DIQ_H.XPT', format='xport', encoding='utf-8')
diq_I = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2015-2016/DIQ_I.XPT', format='xport', encoding='utf-8')
diq_J = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/DIQ_J.XPT', format='xport', encoding='utf-8')

In [48]:
diq_B = diq_B[['SEQN', 'DIQ050', 'DIQ070']]
diq_C = diq_C[['SEQN', 'DIQ050', 'DIQ070']]
diq_D = diq_D[['SEQN', 'DIQ050', 'DID070']]
diq_E = diq_E[['SEQN', 'DIQ050', 'DID070']]
diq_F = diq_F[['SEQN', 'DIQ050', 'DIQ070']]
diq_G = diq_G[['SEQN', 'DIQ050', 'DIQ070']]
diq_H = diq_H[['SEQN', 'DIQ050', 'DIQ070']]
diq_I = diq_I[['SEQN', 'DIQ050', 'DIQ070']]
diq_J = diq_J[['SEQN', 'DIQ050', 'DIQ070']]

diq = pd.concat([diq_B, diq_C, diq_D, diq_E, diq_F, diq_G, diq_H, diq_I, diq_J])
diq.rename(columns={'DIQ050':'taking_insulin', 'DIQ070':'taking_diabetic_pills', 'DID070':'taking_diabetic_pills_D_E'}, inplace=True)

In [49]:
adults = adults.merge(diq, on='SEQN', how='left')

In [50]:
# recode levels for taking insulin / diabetes meds
adults['taking_insulin'] = adults['taking_insulin'].replace(1, 'yes')
adults['taking_insulin'] = adults['taking_insulin'].replace(2, 'no')
adults['taking_insulin'] = adults['taking_insulin'].replace([7, 9], 'unknown')

adults['taking_diabetic_pills'] = adults['taking_diabetic_pills'].replace(1, 'yes')
adults['taking_diabetic_pills'] = adults['taking_diabetic_pills'].replace(2, 'no')
adults['taking_diabetic_pills'] = adults['taking_diabetic_pills'].replace([7, 9], 'unknown')

adults['taking_diabetic_pills_D_E'] = adults['taking_diabetic_pills_D_E'].replace(1, 'yes')
adults['taking_diabetic_pills_D_E'] = adults['taking_diabetic_pills_D_E'].replace(2, 'no')
adults['taking_diabetic_pills_D_E'] = adults['taking_diabetic_pills_D_E'].replace([7, 9], 'unknown')

In [51]:
# create single feature for diabetes (yes/no)
adults['diabetes'] = np.where(adults['fasting_glc_mg_dL'] >= 126, 'yes', 'no')
adults['diabetes'] = np.where(adults['hba1c_percent'] >= 6.5, 'yes', adults['diabetes'])
adults['diabetes'] = np.where(adults['taking_insulin'] == 'yes', 'yes', adults['diabetes'])
adults['diabetes'] = np.where(adults['taking_diabetic_pills'] == 'yes', 'yes', adults['diabetes'])
adults['diabetes'] = np.where(adults['taking_diabetic_pills_D_E'] == 'yes', 'yes', adults['diabetes'])

In [52]:
cov = adults[['SEQN', 'Sex', 'Age', 'Ethnicity', 'Carbohydrate', 'Fiber, total dietary', 'BMI']]

In [53]:
cov = cov.drop_duplicates(subset='SEQN')

In [54]:
print(cov.dropna().shape)
print(cov.shape)
print('no NAs')

(47158, 7)
(47158, 7)
no NAs


### Apply cutoff for percent of calories mapable to DFG2

In [55]:
# calculate the number of calories per ingredient to determine cutoff of percent calories from carbs consumed
adults['carb_cal'] = adults['Carbohydrate'].copy() * 4

In [56]:
# find matches in WWEIA to DFG2
recall_match = adults[adults['ingred_code'].isin(dfg2_data['ingred_code'])]

In [57]:
# calculate the percent of calories from carbohydrates consumed based on the foods that match DFG2
kcal_carb_percent_seqn = recall_match.groupby('SEQN')['carb_cal'].agg(np.sum) / adults.groupby('SEQN')['carb_cal'].agg(np.sum)

In [58]:
kcal_carb_percent_seqn = kcal_carb_percent_seqn.reset_index()

In [59]:
# determine cutoff for percent of calories from carbs consumed
#kcal_carb_percent_seqn = kcal_carb_percent_seqn.reset_index()
kcal_carb_percent_80 = kcal_carb_percent_seqn[kcal_carb_percent_seqn['carb_cal']>0.8000]

In [60]:
# subset the participants by cutoff of 75% calories from carbs consumed
dfg_wweia_80 = recall_match[recall_match['SEQN'].isin(kcal_carb_percent_80['SEQN'])]

In [61]:
# merge wweia data with dfg2 glycan data
dfg_wweia = dfg_wweia_80.merge(dfg2_data, on='ingred_code', how='left')

In [62]:
# sum calories across participants for energy adjustment (per 1000 kcal)
cal_consumed = dfg_wweia.groupby('SEQN')['Energy'].agg(np.sum)

In [63]:
# create index column for groupby level calculations
dfg_wweia['idx'] = dfg_wweia.index

In [64]:
# groupby to sum all intake variables for purpose of energy adjustment
ingred_all = dfg_wweia.groupby(['SEQN', 'diet_wts', 'idx', 'Sex', 'Age', 'BMI', 'diabetes',
                                'ingred_code', 'ingred_desc', 'sample_id', 'simple_name', 
                                'food_group', 'Ethnicity', 'Energy', 'Ingred_consumed_g', 'carb_cal'])[['Carbohydrate',
       'Fiber, total dietary', 'Sugars, total', 'Alcohol',
       'Arabinan', 'Galactan',
       'Xyloglucan', 'Xylan', 'Mannan', 'Beta-Glucan', 'Chitin', 'Starch',
       'Cellulose', 'Arabinoxylan', 'Galactomannan', 'Free_Fructose',
       'Free_Glucose', 'Free_Sucrose', 'Free_Maltose',
       'Free_Lactose(trehalose)', 'Free_Kestose', 'Free_Raffinose',
       'Free_Maltotetraose', 'Free_Stachyose', 'Free_Maltopentaose',
       'Free_Verbascose', 'Free_Maltohexaose', 'Mono_Glucose',
       'Mono_Galactose', 'Mono_Fructose', 'Mono_Xylose', 'Mono_Arabinose',
       'Mono_Fucose', 'Mono_Rhamnose', 'Mono_GalA', 'Mono_Mannose',
       'Mono_Ribose', 't-Glucose', '4-Glucose', '6-Glucose',
       '3-Glucose/3-Galactose', '2-Glucose', '4,6-Glucose', '3,4-Glucose',
       '2,4-Glucose', '3,4,6-Glucose', 't-Galactose', '6-Galactose',
       '4-Galactose', '2-Galactose', '4,6-Galactose', '3,6-Galactose',
       '3,4-Galactose', 't-p-Xylose', '4-p-Xylose', '3-Xylose ', '2-Xylose',
       '3,4-P-Xylose/3,5-Arabinose', '2,4-p-Xylose', 't-f-Arabinose',
       't-p-Arabinose', '5-f-Arabinose', '3-Arabinose', '2-f-Arabinose',
       '2,3-f-Arabinose', 't-Fucose', 't-Rhamnose', '4-Rhamnose ',
       '2-Rhamnose', '2,4-Rhamnose', '4-Glucosamine/GlcNac',
       '3-Glucosamine/GlcNac', 't-Mannose', '4-Mannose', '3-Mannose',
       '2-Mannose', '4,6-Mannose', '2,X-Mannose', '3,4,6-Mannose', 'X-Hexose',
       '2,X,X-Hexose (I)', '2,X,X-Hexose (II)', '2,X,X-Hexose (III)',
       't-Deoxyhexose']].agg(np.sum)

In [65]:
# apply energy adjustment to correct for differences in total enegry consumed (per 1000 kcal for all participants)
ingred_all_adjust = ingred_all.div(cal_consumed, axis = 0, level = 0)
ingred_all_adjust = ingred_all_adjust.mul(1000)
ingred_all_adjust = ingred_all_adjust.reset_index()

In [66]:
# create variable to correct all glycan features for amount of each ingredient consumed
ingred_g = ingred_all_adjust.groupby('idx')['Ingred_consumed_g'].agg(np.sum)

In [67]:
# groupby all intake variables (except carbs and fiber and sugar, already per 100 g) to adjust for amount of ingredient consumed
ingred_all_adjust_g_consumed = ingred_all_adjust.groupby(['SEQN', 'diet_wts', 'idx', 'Sex', 'Age', 'BMI', 'diabetes', 'Ethnicity', 
                                                        'ingred_code', 'ingred_desc', 'sample_id', 'simple_name', 
                                'food_group', 'Energy', 'Ingred_consumed_g', 'carb_cal', 'Carbohydrate',
       'Fiber, total dietary', 'Sugars, total', 'Alcohol'])[[
       'Arabinan', 'Galactan',
       'Xyloglucan', 'Xylan', 'Mannan', 'Beta-Glucan', 'Chitin', 'Starch',
       'Cellulose', 'Arabinoxylan', 'Galactomannan', 'Free_Fructose',
       'Free_Glucose', 'Free_Sucrose', 'Free_Maltose',
       'Free_Lactose(trehalose)', 'Free_Kestose', 'Free_Raffinose',
       'Free_Maltotetraose', 'Free_Stachyose', 'Free_Maltopentaose',
       'Free_Verbascose', 'Free_Maltohexaose', 'Mono_Glucose',
       'Mono_Galactose', 'Mono_Fructose', 'Mono_Xylose', 'Mono_Arabinose',
       'Mono_Fucose', 'Mono_Rhamnose', 'Mono_GalA', 'Mono_Mannose',
       'Mono_Ribose', 't-Glucose', '4-Glucose', '6-Glucose',
       '3-Glucose/3-Galactose', '2-Glucose', '4,6-Glucose', '3,4-Glucose',
       '2,4-Glucose', '3,4,6-Glucose', 't-Galactose', '6-Galactose',
       '4-Galactose', '2-Galactose', '4,6-Galactose', '3,6-Galactose',
       '3,4-Galactose', 't-p-Xylose', '4-p-Xylose', '3-Xylose ', '2-Xylose',
       '3,4-P-Xylose/3,5-Arabinose', '2,4-p-Xylose', 't-f-Arabinose',
       't-p-Arabinose', '5-f-Arabinose', '3-Arabinose', '2-f-Arabinose',
       '2,3-f-Arabinose', 't-Fucose', 't-Rhamnose', '4-Rhamnose ',
       '2-Rhamnose', '2,4-Rhamnose', '4-Glucosamine/GlcNac',
       '3-Glucosamine/GlcNac', 't-Mannose', '4-Mannose', '3-Mannose',
       '2-Mannose', '4,6-Mannose', '2,X-Mannose', '3,4,6-Mannose', 'X-Hexose',
       '2,X,X-Hexose (I)', '2,X,X-Hexose (II)', '2,X,X-Hexose (III)',
       't-Deoxyhexose']].agg(np.sum)

In [68]:
# multiply each glycan feature by amount of ingredient consumed and dividing by 100
ingred_all_adjust_g_consumed = ingred_all_adjust_g_consumed.mul(ingred_g, axis=0, level='idx').div(100, axis=0, level='idx')

In [69]:
# remove columns that aren't needed
ingred_all_adjust_g_consumed.reset_index(inplace=True)
ingred_all_adjust_g_consumed.drop(columns=['idx', 'carb_cal'], inplace=True)

In [70]:
by_seqn = ingred_all_adjust_g_consumed.groupby(['SEQN', 'Age', 'BMI', 'Sex', 'diabetes', 'Ethnicity', 'diet_wts'])[['Carbohydrate',
       'Fiber, total dietary', 'Sugars, total', 'Alcohol',
       'Arabinan', 'Galactan',
       'Xyloglucan', 'Xylan', 'Mannan', 'Beta-Glucan', 'Chitin', 'Starch',
       'Cellulose', 'Arabinoxylan', 'Galactomannan', 'Free_Fructose',
       'Free_Glucose', 'Free_Sucrose', 'Free_Maltose',
       'Free_Lactose(trehalose)', 'Free_Kestose', 'Free_Raffinose',
       'Free_Maltotetraose', 'Free_Stachyose', 'Free_Maltopentaose',
       'Free_Verbascose', 'Free_Maltohexaose', 'Mono_Glucose',
       'Mono_Galactose', 'Mono_Fructose', 'Mono_Xylose', 'Mono_Arabinose',
       'Mono_Fucose', 'Mono_Rhamnose', 'Mono_GalA', 'Mono_Mannose',
       'Mono_Ribose', 't-Glucose', '4-Glucose', '6-Glucose',
       '3-Glucose/3-Galactose', '2-Glucose', '4,6-Glucose', '3,4-Glucose',
       '2,4-Glucose', '3,4,6-Glucose', 't-Galactose', '6-Galactose',
       '4-Galactose', '2-Galactose', '4,6-Galactose', '3,6-Galactose',
       '3,4-Galactose', 't-p-Xylose', '4-p-Xylose', '3-Xylose ', '2-Xylose',
       '3,4-P-Xylose/3,5-Arabinose', '2,4-p-Xylose', 't-f-Arabinose',
       't-p-Arabinose', '5-f-Arabinose', '3-Arabinose', '2-f-Arabinose',
       '2,3-f-Arabinose', 't-Fucose', 't-Rhamnose', '4-Rhamnose ',
       '2-Rhamnose', '2,4-Rhamnose', '4-Glucosamine/GlcNac',
       '3-Glucosamine/GlcNac', 't-Mannose', '4-Mannose', '3-Mannose',
       '2-Mannose', '4,6-Mannose', '2,X-Mannose', '3,4,6-Mannose', 'X-Hexose',
       '2,X,X-Hexose (I)', '2,X,X-Hexose (II)', '2,X,X-Hexose (III)',
       't-Deoxyhexose']].agg(np.sum)

In [71]:
by_seqn.reset_index(inplace=True)

In [72]:
homa = dfg_wweia[['SEQN', 'fasting_glc_mg_dL', 'fasting_ins_uU_mL']]
homa = homa.drop_duplicates(subset='SEQN')
homa = homa.dropna()

In [73]:
homa['homa_ir'] = homa['fasting_ins_uU_mL'] * homa['fasting_glc_mg_dL'] / 405

In [74]:
homa['ir'] = np.where(homa['homa_ir'] >= 2.5, 1, 0)

In [75]:
homa_merge = homa[['SEQN', 'ir']]

In [76]:
cal_carb_final = homa_merge.merge(kcal_carb_percent_80, on='SEQN', how='left')

In [77]:
cal_carb_final['carb_cal'].mean()

0.9169234540021844

In [78]:
seqn_homa = by_seqn.merge(homa_merge, on='SEQN', how='left')

In [79]:
seqn_homa = seqn_homa[seqn_homa['diabetes']=='no'].dropna(subset='ir')

In [80]:
seqn_homa['ir'].value_counts()

0.0    7908
1.0    5753
Name: ir, dtype: int64

In [81]:
seqn_homa.shape

(13661, 92)

In [82]:
seqn_homa.rename(columns={'RIDRETH1': 'Ethnicity'},inplace=True)

In [83]:
seqn_homa.drop(columns='diabetes', inplace=True)

In [84]:
homa_ingredients = ingred_all_adjust_g_consumed[ingred_all_adjust_g_consumed['SEQN'].isin(seqn_homa['SEQN'])]

In [86]:
homa_ingredients[['SEQN', 'ingred_desc', 'simple_name', 'Free_Raffinose', 'Free_Maltohexaose']].to_csv('../output/02/homa_ingredients.csv', index=None)

In [87]:
sample_wt = seqn_homa[['SEQN', 'diet_wts']]
sample_wt.to_csv('../output/02/sample_wt_homa.csv', index=None)

In [88]:
outcome = seqn_homa[['SEQN', 'ir']]
outcome.to_csv('../output/02/outcome_homa.csv', index=None)

In [89]:
seqn_homa.drop(columns=['diet_wts', 'ir']).to_csv('../output/02/dfg2_homa_mod.csv',index=None)

In [90]:
ingred_source = ingred_all_adjust_g_consumed[['SEQN', 'ingred_desc', 'simple_name', 'Free_Raffinose', 'Free_Maltohexaose', '4,6-Glucose', '2-Xylose', 'Free_Verbascose']]

ingred_source = ingred_source[ingred_source['SEQN'].isin(seqn_homa['SEQN'])]

ingred_source.to_csv('../output/02/ingred_source_top_glycans.csv', index=None)

In [91]:
no_alc = seqn_homa[seqn_homa['Alcohol']==0]

In [92]:
no_alc_ingred = ingred_all_adjust_g_consumed[ingred_all_adjust_g_consumed['SEQN'].isin(no_alc['SEQN'])]

In [93]:
no_alc_ingred = no_alc_ingred[['SEQN', 'diet_wts', 'Sex', 'Age', 'BMI', 'diabetes', 'Ethnicity',
       'ingred_code', 'ingred_desc', 'sample_id', 'simple_name', 'Fiber, total dietary', 'Xyloglucan', 'Free_Raffinose', 'Mono_Fucose', '2-Galactose']]

In [94]:
no_alc_ingred.to_csv('../output/02/homa_no_alc_ingred.csv', index=None)

In [95]:
no_alc[['SEQN', 'ir']].to_csv('../output/02/homa_outcome_no_alc.csv', index=None)

In [96]:
no_alc.drop(columns=['Alcohol', 'ir']).to_csv('../output/02/dfg2_homa_no_alc.csv', index=None)